# Resources

Based om Module 2 of the Deep Learning course below (LangGraph)

* [A simple Python implementation of the ReAct pattern for LLMs](https://arc.net/l/quote/duflzttq)
  * Simon Willison Blog Article
* [Deep Learning AI Course, AI Agents with LangGraph](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/1/introduction)

Credits to examples here:
* [OpenAI Documents](https://platform.openai.com/docs/overview)

### Assumptions

* Pythons setup (I created a 3.11 venv)
* OpenAI Key (Granite struggled but perhaps not a fair comparions ollama/grantite q4 8b v `gpt-4o`

In [1]:
import openai
import re
import httpx
import os
import rich
import json
from openai import OpenAI
from agents import Agent, ModelSettings, function_tool,Runner
from rich.pretty import pprint


In [2]:
# Boilerplate for swapping in Granite via ollama
#model = "granite3-dense:8b"
#model = "granite3.1-dense:2b"
#client = OpenAI(
#     base_url='http://localhost:11434/v1',
#     api_key='ollama',
# ) 

from dotenv import load_dotenv
load_dotenv()

model = "gpt-4o"
client = OpenAI() 

In [3]:
# Quick test code - verify LLM conenctivity etc (disable via Raw)

chat_completion = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Write a simple Python example class called User"}],
    temperature=0,
)
print(model)
print(f"{chat_completion.choices[0].message.content}")

gpt-4o
Certainly! Below is a simple example of a Python class called `User`. This class includes basic attributes like `username`, `email`, and `age`, along with a couple of methods to display user information and update the email.

```python
class User:
    def __init__(self, username, email, age):
        self.username = username
        self.email = email
        self.age = age

    def display_user_info(self):
        """Display the user's information."""
        print(f"Username: {self.username}")
        print(f"Email: {self.email}")
        print(f"Age: {self.age}")

    def update_email(self, new_email):
        """Update the user's email address."""
        self.email = new_email
        print(f"Email updated to: {self.email}")

# Example usage:
if __name__ == "__main__":
    # Create a new user
    user1 = User("john_doe", "john@example.com", 30)
    
    # Display user information
    user1.display_user_info()
    
    # Update the user's email
    user1.update_email("john.d

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            #model="gpt-4o",
            model=model,
            temperature=0,
            messages=self.messages)
        return completion.choices[0].message.content

# ReAct Agent Prompt

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run through one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_provision_status:
e.g. get_provision_status: guid
returns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)

log_error:
e.g log_error: status
When a guid has a provision_status ERROR call this with the return value of get_provision_status

log_status:
e.g log_status: status
When a guid does not have an ERROR status call this with the return value of get_provision_status

Example session:

Question: What is the staus of cloud deployment with guid <guid>
Thought: I should look up the status with get_provision_status 
Action: get_provision_status: guid 
PAUSE:

You will be called again with this:

Observation: Guid status "SUCCESS: Completed"

You then call any necessary logging tools before outputing the status:

Answer: Guid status "SUCCESS: Completed"
""".strip()

In [6]:
import random

'''
First of the *fake* functions to test if the LLM/Prompt will ReAct correctly
taking different paths on different results
'''

def get_provision_status(guid):

    #// call to MCP Server AAP2 Controller
    # // foo = bar()
    
    status_messages = [
        "INFO: Initializing",
        "INFO: In progress",
        "ERROR: Failed",
        "ERROR: API Timeout",
        "ERROR: Rate Limited",
        "WARNING: Minor errors",
        "SUCCESS: Completed"
    ]
        # "INFO: Finalizing",
    # return random.choice(f"{guid} status: {status_messages}")
    status = random.choice(status_messages)
    return f"{guid} status: {status}"
    

In [7]:
def log_error(status):
    print(f"{status} Logged stateus to Slack.")
    print(f"{status} Opened Jira Ticket with Status.")
    return 0

def log_status(status):
    print(f"{status} Logged status to Slack.")
    return 0

In [8]:
known_actions = {
   "log_status": log_status,
   "log_error": log_error,
   "get_provision_status": get_provision_status,
}

In [9]:
abot = Agent(prompt)

In [10]:
for i, message in  enumerate(abot.messages):
    if message["role"] != "assistant":
        print(f"Step {i}: App -> LLM:\n")
    else:
        print(f"Step {i}: App <- LLM:\n")
    print(f"Role: {message['role']}\nContent:\n\n{message['content']}\n\n\n")

Step 0: App -> LLM:

Role: system
Content:

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run through one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_provision_status:
e.g. get_provision_status: guid
returns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)

log_error:
e.g log_error: status
When a guid has a provision_status ERROR call this with the return value of get_provision_status

log_status:
e.g log_status: status
When a guid does not have an ERROR status call this with the return value of get_provision_status

Example session:

Question: What is the staus of cloud deployment with guid <guid>
Thought: I should look up the status with get_provision_status 
Action: get_provision_sta

In [11]:
abot = Agent(prompt)
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run through one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nget_provision_status:\ne.g. get_provision_status: guid\nreturns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)\n\nlog_error:\ne.g log_error: status\nWhen a guid has a provision_status ERROR call this with the return value of get_provision_status\n\nlog_status:\ne.g log_status: status\nWhen a guid does not have an ERROR status call this with the return value of get_provision_status\n\nExample session:\n\nQuestion: What is the staus of cloud deployment with guid <guid>\nThought: I should look up the status with get_provision_status \nAction: get

In [12]:
for i, message in  enumerate(abot.messages):
    if message["role"] != "assistant":
        print(f"Step {i}: App -> LLM:\n")
    else:
        print(f"Step {i}: App <- LLM:\n")
    print(f"Role: {message['role']}\nContent:\n\n{message['content']}\n\n\n")

Step 0: App -> LLM:

Role: system
Content:

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run through one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_provision_status:
e.g. get_provision_status: guid
returns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)

log_error:
e.g log_error: status
When a guid has a provision_status ERROR call this with the return value of get_provision_status

log_status:
e.g log_status: status
When a guid does not have an ERROR status call this with the return value of get_provision_status

Example session:

Question: What is the staus of cloud deployment with guid <guid>
Thought: I should look up the status with get_provision_status 
Action: get_provision_sta

# So far a bit to much "Human in the Loop"

Let's automate all this

In [13]:
action_re = re.compile("^Action: (\w+): (.*)$")

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/61/bmwhyqss6rjdhl37b33pbz9m0000gn/T/ipykernel_95638/2177360788.py:1: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile("^Action: (\w+): (.*)$")


In [14]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    print("Step 0")
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            print(f"\nStep {i}")
            # print(f"Actions:\n\n{actions}")
            action, action_inputs = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: -- running {action} {action_inputs}")
            observation = known_actions[action](action_inputs)
            print(f"Observation: {observation}")
            next_prompt = f"Observation: {observation}"
        else:
            return
        

In [15]:
question = """
I have a deployments running with guid: 1adr4 what is its status
"""

query(question)

Step 0
Thought: I should look up the status of the deployment with the given guid using get_provision_status.
Action: get_provision_status: 1adr4 
PAUSE.

Step 1
Observation: 1adr4  status: ERROR: Rate Limited
Thought: Since the status of the deployment is "ERROR: Rate Limited", I should log this error using the log_error action.
Action: log_error: ERROR: Rate Limited
PAUSE.

Step 2
ERROR: Rate Limited Logged stateus to Slack.
ERROR: Rate Limited Opened Jira Ticket with Status.
Observation: 0
Answer: The status of the deployment with guid 1adr4 is "ERROR: Rate Limited".


In [16]:

question = """
I have deployments running with guids: 1adr4, aabf5, 45663, and 45ghb
First get each provision status and log to any services that need to know
Once finished with all deployments output their guids, status, and logging services:

* In a simple table
* As JSON 
"""

query(question, max_turns=10)

Step 0
Thought: I will start by checking the provision status for each of the provided guids one by one. I will log the status accordingly and then output the results in both a table and JSON format. Let's start with the first guid, 1adr4.

Action: get_provision_status: 1adr4
PAUSE

Step 1
Observation: 1adr4 status: SUCCESS: Completed
Thought: The status for guid 1adr4 is "SUCCESS: Completed". I will log this status using the log_status action since it is not an error.

Action: log_status: SUCCESS: Completed
PAUSE

Step 2
SUCCESS: Completed Logged status to Slack.
Observation: 0
Thought: The status for guid 1adr4 has been successfully logged. Now, I will proceed to check the provision status for the next guid, aabf5.

Action: get_provision_status: aabf5
PAUSE

Step 3
Observation: aabf5 status: ERROR: Rate Limited
Thought: The status for guid aabf5 is "ERROR: Rate Limited". I will log this status using the log_error action since it is an error.

Action: log_error: ERROR: Rate Limited
PA

# Function Calling
- Does not work with model = *granite*

In [17]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [18]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)

print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_SLvOnLPjEBc6Bzs872vDv7fl', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function')]


In [19]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])
#print(result)

In [20]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)
print(completion_2.choices[0].message.content)

The current temperature in Paris is 13.4°C.


# Reasoning
## Question: How many other Agent SDKs have this reasoning_effort or something similar exposed?
Is this worth talking about ?

In [21]:
model = "o3-mini"
prompt = """
Write a bash script that takes a matrix represented as a string with 
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = client.chat.completions.create(
    model = model,
    reasoning_effort="medium",
    messages=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

pprint(response)
print('-----------------')
print(response.choices[0].message.content)

ChatCompletion(
│   id='chatcmpl-BIS2qXr1nBLkRTCqfX84HlXF6lX39',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='stop',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content='Below is one way to write a bash script that accepts a matrix string in the format\n\n\u2003\u2003[1,2],[3,4],[5,6]\n\nand prints its transpose (in the same bracketed‐list format). Save the code below (for example, as transpose.sh), make it executable (chmod +x transpose.sh), and run it with a matrix string as its only argument.\n\nBelow is the complete script:\n\n------------------------------------------------------------\n#!/bin/bash\n# Usage: ./transpose.sh \'[1,2],[3,4],[5,6]\'\n\nif [ "$#" -ne 1 ]; then\n  echo "Usage: $0 \'<matrix>\'"\n  exit 1\nfi\n\n# Get the input matrix string.\nmatrix_str="$1"\n\n# Remove an outer leading "[" and trailing "]" if present.\n# For input like "[1,2],[3,4],[5,6]" these are removed.\ntrimmed=$(echo "$matrix_str" | sed \'s/^\\[//; s/\\]$//\')\n\n# Split the matrix into rows.\n# Each row is originally enclosed in brackets, now removed.\n# We assume rows are separated by "],["\nIFS="],[" read -ra rows <<< "$trimmed"\n\n# Determine the number of rows.\nnum_rows=${#rows[@]}\n\n# Get the number of columns from the first row.\nIFS=\',\' read -ra first_row <<< "${rows[0]}"\nnum_cols=${#first_row[@]}\n\n# Create an array to hold the rows of the transposed matrix.\ntransposed=()\n\n# For each column in the original matrix,\n# collect the corresponding element from each row.\nfor (( col=0; col<num_cols; col++ )); do\n  new_row=""\n  for (( row=0; row<num_rows; row++ )); do\n    # Split the current row by comma to get its elements.\n    IFS=\',\' read -ra elements <<< "${rows[row]}"\n    # Trim any accidental spaces.\n    elem=$(echo "${elements[col]}" | xargs)\n    # Build the new row string (comma-separated).\n    if [ "$row" -eq 0 ]; then\n      new_row="$elem"\n    else\n      new_row="$new_row,$elem"\n    fi\n  done\n  # Save the new row.\n  transposed+=("$new_row")\ndone\n\n# Build a string for the transposed matrix in the same format.\nresult=""\nfor (( i=0; i<${#transposed[@]}; i++ )); do\n  if [ "$i" -eq 0 ]; then\n    result="[$(echo "${transposed[i]}")]"\n  else\n    result="$result,[$(echo "${transposed[i]}")]"\n  fi\ndone\n\n# Print the result.\necho "$result"\n\n------------------------------------------------------------\n\nHow it works:\n1. The script first strips any leading or trailing outer brackets.\n2. It splits the string into rows using IFS with the delimiter "],["\n3. For each column index it builds a new “row” gathering that column’s element from every row (which is the definition of the transpose).\n4. Finally, it prints the transposed matrix in the same [a,b],[c,d] format.\n\nExample:\n\n$ ./transpose.sh \'[1,2],[3,4],[5,6]\'\n[1,3,5],[2,4,6]\n\nThis script meets the requirements.',
│   │   │   │   refusal=None,
│   │   │   │   role='assistant',
│   │   │   │   annotations=[],
│   │   │   │   audio=None,
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   created=1743737408,
│   model='o3-mini-2025-01-31',
│   object='chat.completion',
│   service_tier='default',
│   system_fingerprint='fp_617f206dd9',
│   usage=CompletionUsage(
│   │   completion_tokens=1551,
│   │   prompt_tokens=44,
│   │   total_tokens=1595,
│   │   completion_tokens_details=CompletionTokensDetails(
│   │   │   accepted_prediction_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   reasoning_tokens=832,
│   │   │   rejected_prediction_tokens=0
│   │   ),
│   │   prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
│   )
)

-----------------
Below is one way to write a bash script that accepts a matrix string in the format

  [1,2],[3,4],[5,6]

and prints its transpose (in the same bracketed‐list format). Save the code below (for example, as transpose.sh), make it executable (chmod +x transpose.sh), and run it with a matrix string as its only argument.

Below is the complete script:

------------------------------------------------------------
#!/bin/bash
# Usage: ./transpose.sh '[1,2],[3,4],[5,6]'

if [ "$#" -ne 1 ]; then
  echo "Usage: $0 '<matrix>'"
  exit 1
fi

# Get the input matrix string.
matrix_str="$1"

# Remove an outer leading "[" and trailing "]" if present.
# For input like "[1,2],[3,4],[5,6]" these are removed.
trimmed=$(echo "$matrix_str" | sed 's/^\[//; s/\]$//')

# Split the matrix into rows.
# Each row is originally enclosed in brackets, now removed.
# We assume rows are separated by "],["
IFS="],[" read -ra rows <<< "$trimmed"

# Determine the number of rows.
num_rows=${#rows[@]}

# Get

# Conversation States, Memory


# TODO

# Agent
## Agent calling tools
1. Simply demonstrates an agent using a tool.
1. Look at the brevity of the code compared to doing a function calling all on our own.
1. Play with the question that can be asked to agent to see how it can handle questions that may or may not require the tool

In [22]:
model = "gpt-4o"
from agents import Agent, ModelSettings, function_tool, Runner

@function_tool
def get_weather(latitude:str, longitude:str) ->str:
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']
   
agent = Agent(
    name="Blaster",
    instructions="Answer the question asked very precisely. Please think before answering",
    model= model,
    tools=[get_weather],
)

result = await Runner.run(agent, "which is warmer now: Paris or Manila?")
print(result.final_output)
#uncomment the line below to see the detailed interactions
#pprint(result)

Manila is warmer than Paris right now.


## Agents Collaborating
1. Simply demonstrates an agent reviewing the work of another agent - much like a human being.
1. This is one of the primary reasons while the agents can help increase accuracty of the answer and smaller models using agents can outperform larger models without agents.
1. This pattern can be used in lots of scenarios.

In [23]:
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace

"""
This example shows the agents collaborating or one agent reveiwing the work of another and giving feedback. 
The first agent generates an outline for a story.
The second agent judges the outline and provides feedback. 
We loop until the judge is satisfied with the outline.
"""
model = "gpt-4o"

story_outline_generator = Agent(
    name="story_outline_generator",
    instructions=(
        "You generate a very short story outline based on the user's input."
        "If there is any feedback provided, use it to improve the outline."
    ),
    model= model,
)


@dataclass
class EvaluationFeedback:
    feedback: str
    score: Literal["pass", "needs_improvement", "fail"]


evaluator = Agent(
    name="evaluator",
    instructions=(
        "You evaluate a story outline and decide if it's good enough."
        "If it's not good enough, you provide feedback on what needs to be improved."
    ),
    model= model,
    output_type=EvaluationFeedback,
)


In [26]:
msg = input("What kind of story would you like to hear? ")
input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

latest_outline: str | None = None

with trace("Collaboration"):
        i = 0
        while True:
            story_outline_result = await Runner.run(
                story_outline_generator,
                input_items,
            )

            input_items = story_outline_result.to_input_list()
            latest_outline = ItemHelpers.text_message_outputs(story_outline_result.new_items)
            print("Story outline generated")

            evaluator_result = await Runner.run(evaluator, input_items)
            result: EvaluationFeedback = evaluator_result.final_output

            print(f"Evaluator score: {result.score}")

            if result.score == "pass":
                print("Story outline is good enough, exiting.")
                break
            if i == 2:
                print("Maximum number of iterations exceeded, exiting.")
                break
            print("Re-running with feedback")

            input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})
            i += 1

print(f"Final story outline: {latest_outline}")



What kind of story would you like to hear?  humor


Story outline generated
Evaluator score: needs_improvement
Re-running with feedback
Story outline generated
Evaluator score: needs_improvement
Re-running with feedback
Story outline generated
Evaluator score: needs_improvement
Maximum number of iterations exceeded, exiting.
Final story outline: **Title:** "The Great Spaghetti Heist"

**Genre:** Comedy

**Outline:**

**Act 1:**
- *Setting:* A small, quirky Italian town known for its annual spaghetti festival.
- *Characters:*
  - Giovanni, a kind-hearted pizzeria owner, wants to win the festival to honor his late father's legacy as a spaghetti champion.
  - Maria, his sharp-tongued grandmother with a secret past as a spaghetti thief.
  - Luigi, their enthusiastic neighbor with a hidden talent for opera singing.
- Giovanni discovers the town's prized spaghetti recipe has been stolen just days before the festival.

**Act 2:**
- Giovanni, Maria, and Luigi set out on a comedic quest, facing obstacles like accusations from Franco, a rival cla

## Agents routing
1. Simply demonstrates an agent routing work to other agents.
1. This is a very common agentic pattern.
1. Ask the question in German and see what happens! In real life when we use a routing pattern, we must have a fallback agent that gracefully handles all things unknown.

In [27]:

import uuid

#from openai.types.responses import ResponseContentPartDoneEvent, ResponseTextDeltaEvent

from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace

"""
This example shows the handoffs/routing pattern. The triage agent receives the first message, and
then hands off to the appropriate agent based on the language of the request. Responses are
streamed to the user.
"""
model = "gpt-4o"

french_agent = Agent(
    name="french_agent",
    instructions="You only speak French",
    model = model,
)

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You only speak Spanish",
    model = model,
)

english_agent = Agent(
    name="english_agent",
    instructions="You only speak English. Answer the question you recieved.",
    model = model,
)

# experiment by removing  from the instructions the sentence below.
#So answer in English even if you understand the language that is being used.
#And then ask say (German) : Wie geht es dir
know_all_agent = Agent(
    name="know_all_agent",
    instructions="You only speak English. So answer in English even if you understand the language that is being used. \
        State that you do not understand the user question and ask them to repeat it one of the languages you understand. \
        Those languages are English, French and Spanish .",
    model = model,
)

triage_agent = Agent(
    name="triage_agent",
    instructions="Handoff to the appropriate agent based on the language of the request. If you do not know what to do, hand if off to know_all.",
    handoffs=[french_agent, spanish_agent, english_agent, know_all_agent],
    model = model,
)



In [28]:

msg = input("Hi! We speak French, Spanish and English. How can I help? ")
inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

with trace("Router"):
    story_outline_result = await Runner.run(triage_agent,inputs)
    #uncomment this to see the details
    #pprint(story_outline_result)
    print("--------------------------")
    print(story_outline_result.final_output)


Hi! We speak French, Spanish and English. How can I help?  Hey what is the capital of Paris


--------------------------
Paris is the capital of France.


## Agents Deterministic Workflow
1. Simply demonstrates agents calling other agents to complete a well defined workflow.
1. This is a very common agentic pattern.
1. This pattern or its variants can be put to lot of practical use and it could be combined with the collaborative pattern.

In [29]:
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace

"""
This example shows how different agents are used to compelete a deterministic workflow.
In this case it is: 
planner agent -> writer agent -> editor agent 
Given an essay topic, the essay moves through these stages to finally produce an output.
"""
model = "gpt-4o"

@dataclass
class Planner:
    body: str

@dataclass
class Writer:
    body: str

@dataclass
class Editor:
    body: str



planner_agent = Agent(
    name="planner_agent",
    instructions=(
        "Take a user's theme/topic request."
        "Create a brief outline of the essay with points that need to be covered."
        "Make sure that references are given to actual source materials."
    ),
    model= model,
    output_type=Planner,
)



writer_agent = Agent(
    name="writer_agent",
    instructions=(
        "Take the outline given in the input."
        "Expands it into a complete essay."
        "Give adequate references and make sure things are not made up!"
    ),
    model= model,
    output_type=Writer,
)

editor_agent = Agent(
    name="editor_agent",
    instructions=(
        "You Review the draft given in the input."
        "Polish the language, fixes inconsistencies, and improve the flow."
        "And make sure it is logical coherent."
        "Give adequate references and make sure things are not made up!"
        "Return the final story to the user."
    ),
    model= model,
    output_type=Editor,
)



In [30]:
msg = input("Hi! I am AI Researcher. Give me any topic and I will write a well researched essay about it. ")
inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

with trace("Workflow"):
    print("----------Planner Output----------")
    planner_result = await Runner.run(planner_agent,inputs)
    print(planner_result.final_output.body)
    planner_output: Planner = planner_result.final_output
    print("----------Writer Output----------")
    writer_result = await Runner.run(writer_agent,planner_output.body)
    print(writer_result.final_output.body)
    writer_output: Writer = writer_result.final_output
    print("----------Editor Output----------")
    editor_result = await Runner.run(editor_agent,writer_output.body)
    print(editor_result.final_output.body)

Hi! I am AI Researcher. Give me any topic and I will write a well researched essay about it.  AGI


----------Planner Output----------
### Essay Outline on Artificial General Intelligence (AGI)

#### Introduction
- **Definition of AGI**: Explain what AGI is and how it differs from Narrow AI.
- **Importance of AGI**: Discuss the potential impact of AGI on various sectors.
- **Purpose of the Essay**: State the objectives and what the essay will cover.

#### Historical Background
- **Evolution of AI**: Briefly describe the history and evolution of AI leading up to the concept of AGI.
- **Milestones in AI**: Highlight key milestones that have brought AI closer to AGI.
  
#### Characteristics of AGI
- **Human-like Understanding**: Discuss AGI's ability to understand and learn any intellectual task that a human can.
- **Autonomous Decision-Making**: Explore the implications of autonomous decision-making capabilities.

#### Current Approaches to AGI
- **Top-Down vs. Bottom-Up**: Explain different methodologies like symbolism and connectionism.
- **Research Projects**: Discuss ongoing projec

# AFTERWORD
Agent is an extremely powerful construct in the field of Generative AI:
1. You can achieve complex tasks designing appropriate agents and tools.
1. There are known ways by which we can improve accuracy of the output. Much like human beings help check one another's work, agents can do the same.
1. External data retrieval and queries are carried out through the tools.
1. If agent processing needs to be vetted, make sure humans are used (human-in-the-loop) to are used to vet the agent output before it moves to the next step. Really this is no different to how we operate in our real life with human beings - we have review and approval processes etc.